## Classificate all text with Doc2Vec + Logistic Regression

First of all, make the python imports:

In [1]:
# Load for Jupyter Notebook
import sys
sys.path.append('/home/elenaruiz/Documents/TFG/FNC')

In [89]:
import pandas as pd
import numpy as np
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.model_selection import train_test_split

from src.utils import io
from src.fake_news_detector.core.nlp import clean_text as ct

### 1. Load dataset (tmp.json)

The file `tmp.json` contains all raw data that has to be tokenize

In [37]:
articles = io.read_json_file('/home/elenaruiz/Documents/TFG/FNC/src/data/tmp.json')
df = pd.DataFrame(data=articles['articles']) # Put in pandas dataframe

This dataset contains:
    - Title
    - Subtitle
    - Text (list of pharagraphs)
    - Fake (if it is fake)
    - Url

In [38]:
df.columns

Index(['fake', 'subtitle', 'text', 'title', 'url'], dtype='object')

### 2. Join and clean raw text

Tokenize in words. For each article store list of Tokens

In [39]:
corpus = []
i = 0
for _, row in df.iterrows():
    x = ct.clean_text_by_word(row['title'], True)
    y = ct.clean_text_by_word(row['subtitle'], True)
    z = []
    for sent in row['text']:
        z += ct.clean_text_by_word(sent, True)
    corpus.append(x + y + z)
    i = i + 1
df['corpus'] = corpus

### 3. Split datasets
Split datasets in two parts: Train and Test.

In [40]:
X_train, X_test = train_test_split(df, random_state=0)

### 4. Build Doc2Vec with training data

Build taggeds documents in other to insert them on Doc2Vec

In [62]:
tagged_data = []
for _, row in X_train.iterrows():
    x = int(row['fake'] == True)
    tagged_data.append(TaggedDocument(row['corpus'],str(x)))

#### 4.1 Create Doc2Vec
Create default model with our configuration

In [63]:
# DOC2VEC Configuration
max_epochs = 100
vec_size = 20
alpha = 0.025
model = Doc2Vec(alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

#### 4.2 Build vocabulary
Build our vocabulary:

In [64]:
model.build_vocab(tagged_data)

#### 4.3 Doc2Vec model

In [65]:
for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

/home/elenaruiz/Documents/TFG/FNC/env/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


TypeError: list indices must be integers or slices, not str